In [1]:
from os.path import isfile, join, exists
from os import listdir, makedirs
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
from brainpipe.system import study
from scipy import stats
from itertools import product

### Mean correlation by patient for all significant electrodes // By freq

In [9]:
th, feat, corr = '0.01', 'pow', False
###############################################################################
st = study('Olfacto')
path_npz = join(st.path, 'figure/0_Classif_Power_R_rest_EpiPerf_LowHigh_200perm/')
path_mask = join(path_npz, 'masks_stat/')
path2save = join(path_npz, 'Correlations'+th+'/')
npz_form = join(path_npz, '{}_sources_{}_{}_low_high_sel_physFT.npz')
masks_vis_form = join(path_mask, '{}_mask_stat_{}_minwin{}_th{}.npy')
###############################################################################
if not exists(path2save):
    makedirs(path2save)
############################################################################### 
subjects = ['CHAF','FERJ','LEFC','SEMC','PIRJ','MICP','VACJ']
su_codes = ['S0','S1','S2','S3','S4','S5','S6']

# Load mean results by subjects
freqs = ['0_VLFC', '1_delta', '2_theta', '3_alpha', '4_beta','5_gamma1','6_gamma2']
method, win = ['s_Mail_RL'], 1

for freq in freqs:
    # Behavioral Performance
    scores_tot = [12.3,2.4,8.0,2.1,7.7,6.3,8.3]
    scores_low = [2.3,2.0,0.8,1.6,1.2,1.8,1.7]
    scores_high = [22.3,2.8,15.2,2.9,14.2,8.9,15.5]
    scores_diff = [scores_high[i]-scores_low[i] for i in range(len(scores_high))]
    
    arch_sig = np.load(npz_form.format('All_subjects',freq, 'odor'))
    if exists(masks_vis_form.format('All_subjects',freq,str(win),th)):
        mask = np.load(masks_vis_form.format('All_subjects',freq,str(win),th))
        mask = np.logical_not(mask) #inverse True False
        su_codes = arch_sig['su_codes'][mask]
        s_da, s_power = arch_sig['s_da'], ((arch_sig['s_elec_pow1']-arch_sig['s_elec_pow0'])/arch_sig['s_elec_pow0'])*100
        s_time = np.arange(-500,2000,100)
        print('original shape',s_da.shape, s_power.shape,s_time.shape)
        #Select the max AUC and corresponding power change for all electrodes
        da_all, pow_all,time_all = np.array([]), np.array([]),np.array([])
        for elec in range(s_da.shape[0]):
            da_elec = s_da[elec][5:30]
            idx = [i for i,j in enumerate(da_elec) if j ==max(da_elec)][0]
            pow = s_power[elec][idx]
            da = da_elec[idx]
            time = s_time[idx]
            pow_all = np.hstack((pow_all,pow)) if np.size(pow_all) else pow
            da_all = np.hstack((da_all, da)) if np.size(da_all) else da
            time_all = np.hstack((time_all,time)) if np.size(time_all) else time
        print('max AUC shape',da_all.shape, pow_all.shape,time_all.shape)
        #Mask non-significant electrodes
        da_all,pow_all,time_all = da_all[mask], pow_all[mask], time_all[mask]
        #Create an array of scores by patient and electrodes and masked it
        #scores_elecs = [scores_tot[int(k[1])] for k in sorted(subjects)]
        
        #Create an array of mean values by subject
        da_by_su, pow_by_su, time_by_su = np.array([]), np.array([]),np.array([])
        for s,su in enumerate(subjects):
            da_su = np.mean(da_all[np.where(su_codes=='S'+str(s))])
            pow_su = np.mean(pow_all[np.where(su_codes=='S'+str(s))])
            time_su = np.mean(time_all[np.where(su_codes=='S'+str(s))])
            da_by_su = np.hstack((da_by_su,da_su)) if np.size(da_by_su) else da_su
            pow_by_su = np.hstack((pow_by_su,pow_su)) if np.size(pow_by_su) else pow_su
            time_by_su = np.hstack((time_by_su,time_su)) if np.size(time_by_su) else time_su
        print(da_by_su,pow_by_su,time_by_su)
        #remove scores values when patient not included
        scores_diff = np.array(scores_diff)[np.where(~np.isnan(da_by_su))]
        scores_tot = np.array(scores_tot)[np.where(~np.isnan(da_by_su))]
        print(scores_diff, scores_tot)
        #remove nan values
        da_by_su = da_by_su[np.logical_not(np.isnan(da_by_su))] 
        pow_by_su = pow_by_su[np.logical_not(np.isnan(pow_by_su))] 
        time_by_su = time_by_su[np.logical_not(np.isnan(time_by_su))]
        print(da_by_su.shape,pow_by_su.shape,time_by_su.shape)

        if da_by_su.shape[0] >=4:
            #Plot all correlations
            scores, features = [scores_tot,scores_diff], [da_by_su,pow_by_su,time_by_su]
            scores_names, feats_names= ['tot_EM','diff_EM',],['da_by_su','pow_by_su','time_by_su']
            for i,score in enumerate(scores):
                for j,feat in enumerate(features):
                    R, p = stats.pearsonr(score,feat)
                    R, p = round(R,2), round(p,3)
                    if p < 0.05:
                        print('stats for',freq,scores_names[i],feats_names[j],R,p)
                        fig, ax = plt.subplots()
                        plt.title('Correlation btw '+scores_names[i]+' and '+feats_names[j]+' for '+freq)
                        plt.xlabel(scores_names[i]), plt.ylabel(feats_names[j])
                        anchored_text = AnchoredText('R coeff = %s, pval = %s' % (R, p), loc=2)
                        ax.add_artist(anchored_text)
                        fit = np.polyfit(score, feat, deg=1)
                        ax.plot(sorted(score), fit[0] * np.array(sorted(score)) + fit[1], color='red')
                        ax.scatter(score, feat, color = 'dodgerblue')
                        plt.savefig(path2save+'Correlation_'+freq+'_'+scores_names[i]+'_'+feats_names[j]+'.png')
                        plt.clf()
                        plt.close()

-> Olfacto loaded
original shape (405, 22) (405, 22) (25,)
max AUC shape (405,) (405,) (405,)
[ nan  nan  nan  nan  nan  nan  nan] [ nan  nan  nan  nan  nan  nan  nan] [ nan  nan  nan  nan  nan  nan  nan]
[] []
(0,) (0,) (0,)
original shape (405, 22) (405, 22) (25,)
max AUC shape (405,) (405,) (405,)
[        nan         nan  0.95666667  0.905       0.90583333  0.91166667
         nan] [          nan           nan  -60.9262508   -32.08509752  -46.87145886
  162.34479526           nan] [  nan   nan  900.  900.  500.  900.   nan]
[ 14.4   1.3  13.    7.1] [ 8.   2.1  7.7  6.3]
(4,) (4,) (4,)
original shape (405, 22) (405, 22) (25,)
max AUC shape (405,) (405,) (405,)
[ 0.915       0.87583333  0.90166667  0.68               nan  0.91333333
         nan] [-54.79379367  91.96319126 -17.7385449  -23.33789093          nan
  80.42116583          nan] [ 900. -500. -200. -300.   nan  700.   nan]
[ 20.    0.8  14.4   1.3   7.1] [ 12.3   2.4   8.    2.1   6.3]
(5,) (5,) (5,)
original shape (405, 22

/home/karim/anaconda3/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/karim/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


max AUC shape (405,) (405,) (405,)
[        nan  0.90333333  0.84               nan         nan  0.88833333
         nan] [         nan  48.78615724  37.05618514          nan          nan
  -6.81886483          nan] [  nan -200. -200.   nan   nan  675.   nan]
[  0.8  14.4   7.1] [ 2.4  8.   6.3]
(3,) (3,) (3,)
original shape (405, 22) (405, 22) (25,)
max AUC shape (405,) (405,) (405,)
[ 0.92666667  0.96166667  0.91333333  0.91666667         nan  0.90166667
         nan] [-14.73738109  62.61646836 -33.69805617 -10.60923777          nan
  10.24039132          nan] [-100.  900.  500. -300.   nan  800.   nan]
[ 20.    0.8  14.4   1.3   7.1] [ 12.3   2.4   8.    2.1   6.3]
(5,) (5,) (5,)
original shape (405, 22) (405, 22) (25,)
max AUC shape (405,) (405,) (405,)
[        nan  0.86541667         nan  0.81083333         nan  0.95666667
         nan] [         nan -16.21112621          nan -11.72333165          nan
  34.11567677          nan] [  nan  600.   nan  250.   nan -150.   nan]
[ 0.8  

### Correlations btw significant electrodes and memory score // By freq (no mean)

In [6]:
th, feat, corr = 'try', 'pow', False
###############################################################################
st = study('Olfacto')
path_npz = join(st.path, 'figure/0_Classif_Power_R_EpiPerf_LowHigh_200perm/')
path_mask = join(path_npz, 'masks_stat/')
path2save = join(path_npz, 'Correlations'+th+'/')
npz_form = join(path_npz, '{}_sources_{}_{}_low_high_sel_physFT.npz')
masks_vis_form = join(path_mask, '{}_mask_stat_{}_minwin{}_th{}.npy')
###############################################################################
if not exists(path2save):
    makedirs(path2save)
############################################################################### 
subjects = ['CHAF','FERJ','LEFC','SEMC','PIRJ','VACJ']
su_codes = ['S0','S1','S2','S3','S4','S5','S6']

# Load mean results by subjects
freqs = ['0_VLFC', '1_delta', '2_theta', '3_alpha', '4_beta','5_gamma1','6_gamma2']
method, win = ['s_Mail_RL'], 1

for freq in freqs:
    # Behavioral Performance
    scores_tot = [12.3,2.4,8.0,2.1,7.7,6.3,8.3]
    scores_low = [2.3,2.0,0.8,1.6,1.2,1.8,1.7]
    scores_high = [22.3,2.8,15.2,2.9,14.2,8.9,15.5]
    scores_diff = [scores_high[i]-scores_low[i] for i in range(len(scores_high))]
    
    arch_sig = np.load(npz_form.format('All_subjects',freq, 'odor'))
    mask = np.load(masks_vis_form.format('All_subjects',freq,str(win),th))
    mask = np.logical_not(mask) #inverse True False
    su_codes = arch_sig['su_codes'][mask]
    s_da, s_power = arch_sig['s_da'], ((arch_sig['s_elec_pow1']-arch_sig['s_elec_pow0'])/arch_sig['s_elec_pow0'])*100
    s_time = np.arange(-500,2000,100)
    #print('original shape',s_da.shape, s_power.shape,s_time.shape)
    #Select the max AUC and corresponding power change for all electrodes
    da_all, pow_all,time_all = np.array([]), np.array([]),np.array([])
    for elec in range(s_da.shape[0]):
        da_elec = s_da[elec][5:30]
        idx = [i for i,j in enumerate(da_elec) if j ==max(da_elec)][0]
        pow = s_power[elec][idx]
        da = da_elec[idx]
        time = s_time[idx]
        pow_all = np.hstack((pow_all,pow)) if np.size(pow_all) else pow
        da_all = np.hstack((da_all, da)) if np.size(da_all) else da
        time_all = np.hstack((time_all,time)) if np.size(time_all) else time
    #print('max AUC shape',da_all.shape, pow_all.shape,time_all.shape)
    #Mask non-significant electrodes
    da_all,pow_all,time_all = da_all[mask], pow_all[mask], time_all[mask]
    print(pow_all.shape, pow_all)
    
    scores, features = [scores_tot], [da_all,pow_all,time_all]
    scores_names, feats_names= ['tot_EM'],['da','pow','time']
    for i,score in enumerate(scores):
        #Create an array of scores by patient and electrodes and masked it
        scores_elecs = [score[int(k[1])] for k in sorted(su_codes)]
        for j,feat in enumerate(features):
            R, p = stats.pearsonr(scores_elecs,feat)
            R, p = round(R,3), round(p,6)
            print('score',freq,R,p)
            if p <= 0.05:
                print('score',freq,'mask',len(np.where(mask==1)[0]),scores_names[i], feat.shape, feats_names[j],R,p)
                fig, ax = plt.subplots()
                plt.title('Correlation all // '+scores_names[i]+' & '+feats_names[j]+'-'+freq+' n_points = '+str(len(np.where(mask==1)[0])))
                plt.xlabel(scores_names[i]), plt.ylabel(feats_names[j])
                anchored_text = AnchoredText('R coeff = %s, pval = %s' % (R, p), loc=2)
                ax.add_artist(anchored_text)
                fit = np.polyfit(scores_elecs, feat, deg=1)
                ax.plot(sorted(scores_elecs), fit[0] * np.array(sorted(scores_elecs)) + fit[1], color='red')
                ax.scatter(scores_elecs, feat, color = 'dodgerblue')
                plt.savefig(path2save+'Correlation_'+freq+'_'+scores_names[i]+'_'+feats_names[j]+'.png')
                plt.clf()
                plt.close()

-> Olfacto loaded
(39,) [  76.31549629   98.95358996  -26.18620574  -56.6469241   130.22178476
  -20.53681158    7.92282147   28.55107728  339.84402642   51.35041371
  -32.39974111   20.52513301    5.19490303  -56.53579068  107.57504207
   -7.65777645  -53.64496201  -49.83819009  -60.80070517  -20.24317289
   68.26496609  100.76620206  -17.6205447   -35.39992873  -34.29767685
   71.02938737  -54.92290697  -67.28298651  -28.34905958  -74.31390582
  -86.04716816  -23.36765035   67.04584614   73.95188667  -48.97946241
  158.22841559  133.37756919  -68.54371834  177.47946093]
score 0_VLFC -0.167 0.308359
score 0_VLFC 0.325 0.043388
score 0_VLFC mask 39 tot_EM (39,) pow 0.325 0.043388
score 0_VLFC -0.19 0.247174
(50,) [  59.19641084  148.67964164   73.92688924   99.49339843   20.68418373
   -9.5916554   -47.01338252   -9.78266995   64.7893337    75.07143949
  -67.90631809   71.50551558  -59.55356444  -53.85833768    4.69096836
  -54.24341523  -24.15227337  -48.19923915  -65.08964519   61.09

### Correlation btw memory score and classif in specific regions
    Frontal vs Temporal regions

In [7]:
th, feat, corr = 'try', 'pow', False
###############################################################################
st = study('Olfacto')
path_npz = join(st.path, 'figure/0_Classif_Power_R_EpiPerf_LowHigh_200perm/')
path_mask = join(path_npz, 'masks_stat/')
path2save = join(path_npz, 'Correlations'+th+'/')
npz_form = join(path_npz, '{}_sources_{}_{}_low_high_sel_physFT.npz')
masks_vis_form = join(path_mask, '{}_mask_stat_{}_minwin{}_th{}.npy')
###############################################################################
if not exists(path2save):
    makedirs(path2save)
############################################################################### 
subjects = ['CHAF','FERJ','LEFC','SEMC','PIRJ','MICP','VACJ']
su_codes = ['S0','S1','S2','S3','S4','S5','S6']

# Load mean results by subjects
freqs = ['0_VLFC', '1_delta', '2_theta', '3_alpha', '4_beta','5_gamma1','6_gamma2']
method, win = ['s_Mail_RL'], 1

# List of rois to test
rois_list = {'Frontal':['ACC','IFG','MFG','OFC','SFG','Ins'],
            'Olf':['Amg','pPirT','Amg-PirT','Ins','OFC'],
            'MTL':['HC','PHG'],
            'Temporal':['FuG','HC','PHG','ITG','MTG','STG','Amg','pPirT','Amg-PirT'],
            'all':['ACC','IFG','MFG','OFC','SFG','Ins','FuG','HC','PHG','ITG','MTG','STG','Amg','pPirT','Amg-PirT']}

for freq in freqs:
    # Behavioral Performance
    scores_tot = [12.3,2.4,8.0,2.1,7.7,6.3,8.3]
    scores_low = [2.3,2.0,0.8,1.6,1.2,1.8,1.7]
    scores_high = [22.3,2.8,15.2,2.9,14.2,8.9,15.5]
    scores_diff = [scores_high[i]-scores_low[i] for i in range(len(scores_high))]
    
    arch_sig = np.load(npz_form.format('All_subjects',freq, 'odor'))
    mask = np.load(masks_vis_form.format('All_subjects',freq,str(win),th))
    mask = np.logical_not(mask) #inverse True False
    s_da, s_power = arch_sig['s_da'], ((arch_sig['s_elec_pow1']-arch_sig['s_elec_pow0'])/arch_sig['s_elec_pow0'])*100
    s_time = np.arange(-500,2000,100)
    su_codes = arch_sig['su_codes'][mask]
    s_scores = [scores_tot[int(k[1])] for k in sorted(su_codes)]
    print(s_scores,len(s_scores))
    #print('original shape',s_da.shape, s_power.shape,s_time.shape)
    
    #Select the max AUC and corresponding power change for all electrodes
    da_all, pow_all,time_all = np.array([]), np.array([]),np.array([])
    for elec in range(s_da.shape[0]):
        da_elec = s_da[elec][5:30]
        idx = [i for i,j in enumerate(da_elec) if j ==max(da_elec)][0]
        pow = s_power[elec][idx]
        da = da_elec[idx]
        time = s_time[idx]
        pow_all = np.hstack((pow_all,pow)) if np.size(pow_all) else pow
        da_all = np.hstack((da_all, da)) if np.size(da_all) else da
        time_all = np.hstack((time_all,time)) if np.size(time_all) else time
    #print('max AUC shape',da_all.shape, pow_all.shape,time_all.shape)
    #Mask non-significant electrodes
    da_all,pow_all,time_all = da_all[mask], pow_all[mask], time_all[mask]
    
    # Select only elecs in specific rois
    for region in rois_list:
        s_Mai_RL = arch_sig['s_MAI_RL'][mask]
        da_rois, pow_rois, time_rois, scores_elecs = np.array([]), np.array([]),np.array([]),np.array([])
        for roi in rois_list[region]:
            #Create an array of mean values by subject
            da_roi = da_all[np.where(s_Mai_RL==roi)]
            pow_roi = pow_all[np.where(s_Mai_RL==roi)]
            time_roi = time_all[np.where(s_Mai_RL==roi)]
            score_elec = [s_scores[i] for i in range(len(s_scores)) if s_Mai_RL[i]==roi]
            da_rois = np.hstack((da_rois,da_roi)) if np.size(da_rois) else da_roi
            pow_rois = np.hstack((pow_rois,pow_roi)) if np.size(pow_rois) else pow_roi
            time_rois = np.hstack((time_rois,time_roi)) if np.size(time_rois) else time_roi
            scores_elecs = np.hstack((scores_elecs,score_elec)) if np.size(scores_elecs) else score_elec
        print('features by roi',freq, region, pow_rois.shape)
        
        features = [da_rois,pow_rois,time_rois]
        score_name, feats_names= 'tot_EM',['da','pow','time']
        for j,feat in enumerate(features):
            R, p = stats.pearsonr(scores_elecs,feat)
            R, p = round(R,3), round(p,6)
            if p <= 0.05 and feat.shape[0] >=10:
                print(region,'score',freq,'mask',len(np.where(mask==1)[0]),scores_names[i], feat.shape, feats_names[j],R,p)
                print(len(scores_elecs))
                fig, ax = plt.subplots()
                plt.title('Correlation '+region+' // '+scores_names[i]+' & '+feats_names[j]+'-'+freq+' n_points = '+str(feat.shape[0]))
                plt.xlabel(scores_names[i]), plt.ylabel(feats_names[j])
                anchored_text = AnchoredText('R coeff = %s, pval = %s' % (R, p), loc=2)
                ax.add_artist(anchored_text)
                fit = np.polyfit(scores_elecs, feat, deg=1)
                ax.plot(sorted(scores_elecs), fit[0] * np.array(sorted(scores_elecs)) + fit[1], color='red')
                ax.scatter(scores_elecs, feat, color = 'dodgerblue')
                plt.savefig(path2save+'Correlation_'+freq+'_'+scores_names[i]+'_'+feats_names[j]+'_'+region+'.png')
                plt.clf()
                plt.close()

-> Olfacto loaded
[12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 8.0, 8.0, 8.0, 8.0, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 7.7, 7.7, 7.7, 7.7, 6.3, 6.3, 6.3, 6.3, 6.3] 39
features by roi 0_VLFC Olf (8,)
features by roi 0_VLFC MTL (6,)
features by roi 0_VLFC Frontal (25,)
Frontal score 0_VLFC mask 39 tot_EM (25,) pow 0.433 0.030443
25
features by roi 0_VLFC all (39,)
all score 0_VLFC mask 39 tot_EM (39,) pow 0.325 0.043388
39
features by roi 0_VLFC Temporal (14,)
[12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 7.7, 7.7, 7.7, 6.3, 6.3, 6.3, 6.3, 6.3, 6.3, 6.3, 6.3, 6.3, 6.3] 50
features by roi 1_delta Olf (9,)
features by roi 1_delta MTL (7,)
features by roi 1_delta Frontal (30,)
features by roi 1_delta all (50,)
features by roi 1_delta Temporal (20,)
Temporal score 1_delta mask 50 tot

/home/karim/anaconda3/lib/python3.5/site-packages/scipy/stats/stats.py:3003: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


features by roi 3_alpha Temporal (25,)
[12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 7.7, 7.7, 7.7, 7.7, 7.7, 6.3, 6.3, 6.3, 6.3, 6.3, 6.3, 6.3, 6.3, 6.3, 6.3, 6.3, 6.3, 6.3] 66
features by roi 4_beta Olf (12,)
Olf score 4_beta mask 66 tot_EM (12,) time -0.891 9.9e-05
12
features by roi 4_beta MTL (6,)
features by roi 4_beta Frontal (38,)
Frontal score 4_beta mask 66 tot_EM (38,) time -0.338 0.037795
38
features by roi 4_beta all (66,)
all score 4_beta mask 66 tot_EM (66,) time -0.248 0.044486
66
features by roi 4_beta Temporal (28,)
[12.3, 12.3, 12.3, 12.3, 12.3, 12.3, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 7.7, 7.7, 7.7, 7.7, 7.7, 6.3, 6.3, 6.3, 6.3, 6.3, 6.3, 6.

### Correlation btw power change and memory perf in specific regions
    Where at least 5 patients have significant electrodes

In [ ]:
th, feat, corr = '0.01', 'pow', False
###############################################################################
st = study('Olfacto')
path_npz = join(st.path, 'figure/0_Classif_Power_E_EpiPerf_LowHigh_1000perm/')
path_mask = join(path_npz, 'masks_visbrain'+th+'/')
path2save = join(path_npz, 'Correlations'+th+'/')
npz_form = join(path_npz, '{}_sources_{}_{}_low_high_sel_physFT.npz')
masks_vis_form = join(path_mask, '{}_mask_min{}_{}_minwin{}_th{}_corr.npy') if corr == True else join(path_mask, '{}_mask_min{}_{}_minwin{}_th{}.npy')
f_form_save = join(path2save, 'Correl_{}_signif_min{}_'+feat+'_{}_th{}_corr.png') if corr == True else join(path2save, 'Correl_{}_signif_min{}_'+feat+'_{}_th{}.png')
###############################################################################

subjects = ['CHAF','FERJ','LEFC','SEMC','PIRJ','MICP','VACJ']
su_codes = ['S0','S1','S2','S3','S4','S5','S6']

# Load mean results by subjects
freqs = ['0_VLFC', '1_delta', '2_theta', '3_alpha', '4_beta','5_gamma1','6_gamma2']
method, win = ['s_Mail_RL'], 1

for freq in freqs:
    # Power and classif results
    arch_sig = np.load(npz_form.format('All_subjects',freq, 'odor'))
    mask = np.load(masks_vis_form.format('MAI_RL',freq,str(win),th))
    mask = np.logical_not(mask) #inverse True False
    su_codes = arch_sig['su_codes'][mask]
    s_Mai_RL = arch_sig['s_MAI_RL'][mask]
    rois_list = np.unique(s_Mai_RL)
    print(rois_list)
    s_da, s_power = arch_sig['s_da'], ((arch_sig['s_elec_pow1']-arch_sig['s_elec_pow0'])/arch_sig['s_elec_pow0'])*100
    s_time = np.arange(-1000,2500,100)
    print(s_da.shape, s_power.shape,s_time.shape)
    #Select the max AUC and corresponding power change for all electrodes
    da_all, pow_all,time_all = np.array([]), np.array([]),np.array([])
    for elec in range(s_da.shape[0]):
        da_elec = s_da[elec]
        idx = [i for i,j in enumerate(da_elec) if j ==max(da_elec)][0]
        pow = s_power[elec][idx]
        da = da_elec[idx]
        time = s_time[idx]
        pow_all = np.hstack((pow_all,pow)) if np.size(pow_all) else pow
        da_all = np.hstack((da_all, da)) if np.size(da_all) else da
        time_all = np.hstack((time_all,time)) if np.size(time_all) else time
    print(da_all.shape, pow_all.shape,time_all.shape)
    #Mask non-significant electrodes
    da_all,pow_all,time_all = da_all[mask], pow_all[mask], time_all[mask]
    #print(da_all.shape, pow_all.shape)
    for roi in rois_list:
        # Behavioral Performance
        scores_tot = [12.3,2.4,8.0,2.1,7.7,6.3,8.3]
        scores_low = [2.3,2.0,0.8,1.6,1.2,1.8,1.7]
        scores_high = [22.3,2.8,15.2,2.9,14.2,8.9,15.5]
        scores_diff = [scores_high[i]-scores_low[i] for i in range(len(scores_high))]
        #Create an array of mean values by subject
        da_by_su, pow_by_su, time_by_su = np.array([]), np.array([]),np.array([])
        for s,su in enumerate(subjects):
            da_su = np.mean(da_all[np.where((su_codes=='S'+str(s))&(s_Mai_RL==roi))])
            pow_su = np.mean(pow_all[np.where((su_codes=='S'+str(s))&(s_Mai_RL==roi))])
            time_su = np.mean(time_all[np.where((su_codes=='S'+str(s))&(s_Mai_RL==roi))])
            da_by_su = np.hstack((da_by_su,da_su)) if np.size(da_by_su) else da_su
            pow_by_su = np.hstack((pow_by_su,pow_su)) if np.size(pow_by_su) else pow_su
            time_by_su = np.hstack((time_by_su,time_su)) if np.size(time_by_su) else time_su
        #remove scores values when patient not included
        scores_diff = np.array(scores_diff)[np.where(~np.isnan(da_by_su))]
        scores_tot = np.array(scores_tot)[np.where(~np.isnan(da_by_su))]
        print(scores_diff, scores_tot)
        #remove nan values
        da_by_su = da_by_su[np.logical_not(np.isnan(da_by_su))] 
        pow_by_su = pow_by_su[np.logical_not(np.isnan(pow_by_su))] 
        time_by_su = time_by_su[np.logical_not(np.isnan(time_by_su))]
        print(da_by_su,pow_by_su,time_by_su)
        
        if da_by_su.shape[0] >= 4:
            #Plot all correlations
            scores, features = [scores_tot,scores_diff], [da_by_su,pow_by_su,time_by_su]
            scores_names, feats_names= ['tot_EM','diff_EM',],['da_by_su','pow_by_su','time_by_su']
            for i,score in enumerate(scores):
                for j,feat in enumerate(features):
                    R, p = stats.pearsonr(score,feat)
                    R, p = round(R,2), round(p,3)
                    if p <= 0.07:
                        print('stats for',freq,scores_names[i],feats_names[j],R,p)
                        fig, ax = plt.subplots()
                        plt.title('Correlation btw '+scores_names[i]+' and '+feats_names[j]+' for '+freq)
                        plt.xlabel(scores_names[i]), plt.ylabel(feats_names[j])
                        anchored_text = AnchoredText('R coeff = %s, pval = %s' % (R, p), loc=2)
                        ax.add_artist(anchored_text)
                        fit = np.polyfit(score, feat, deg=1)
                        ax.plot(sorted(score), fit[0] * np.array(sorted(score)) + fit[1], color='red')
                        ax.scatter(score, feat, color = 'dodgerblue')
                        plt.savefig(path2save+'Correlation_'+freq+'_'+roi+'_'+scores_names[i]+'_'+feats_names[j]+'_corr.png') if corr==True else plt.savefig(path2save+'Correlation_'+freq+'_'+roi+'_'+scores_names[i]+'_'+feats_names[j]+'.png')
                        plt.clf()
                        plt.close()
                    else:
                        continue
        else:
            continue

### Correlations btw memory perf and encoding strategy

In [ ]:
th, bsl, feat = '01', 'None', 'pow'
###############################################################################
st = study('Olfacto')
path_npz = join(st.path, 'figure/0_Classif_Power_E_EpiPerf_LowHigh_'+bsl+'/')
path_mask = join(path_npz, 'masks_visbrain'+th+'/')
path2save = join(path_npz, 'Correlations/')
npz_form = join(path_npz, '{}_sources_{}_{}_low_high_sel_physFT_{}.npz')
masks_vis_form = join(path_mask, '{}_mask_min{}_{}_minwin{}_th{}.npy')
f_form_save = join(path2save, 'Correl_{}_signif_min{}_'+feat+'_bsl_{}_th{}.png')
###############################################################################

subjects = ['CHAF','FERJ','LEFC','SEMC','PIRJ','MICP','VACJ']
su_codes = ['S0','S1','S2','S3','S4','S5','S6']

# Memory Performance
scores_tot = [12.3,2.4,8.0,2.1,7.7,6.3,8.3]
scores_low = [2.3,2.0,0.8,1.6,1.2,1.8,1.7]
scores_high = [22.3,2.8,15.2,2.9,14.2,8.9,15.5]
scores_diff = [scores_high[i]-scores_low[i] for i in range(len(scores_high))]

#Encoding Performance
count = [8.9,9.9,10.9,7.4,4.9,3.8,6.8]
delta = [16.5,19.7,8.7,16.8,30.3,35.3,29.8]
dur = [2.9,1.3,1.3,1.6,1.7,0.8,1.4]
vol = [9.3,1.5,2.4,3.4,5.8,1.1,3.1]
ampl = [4.5,2.0,3.0,3.5,4.5,2.3,3.2]
    
#Plot all correlations
scores, features = [scores_tot,scores_diff], [count,delta,dur,vol,ampl]
scores_names, feats_names= ['tot_EM','diff_EM'],['count','delta','dur','vol','ampl']
for i,score in enumerate(scores):
    for j,feat in enumerate(features):
        R, p = stats.pearsonr(score,feat)
        R, p = round(R,2), round(p,3)
        print(p)
        if p <= 0.06:
            print('stats for',scores_names[i],feats_names[j],R,p)
            fig, ax = plt.subplots()
            plt.title('Correlation btw '+scores_names[i]+' and '+feats_names[j]+' for '+freq)
            plt.xlabel(scores_names[i]), plt.ylabel(feats_names[j])
            anchored_text = AnchoredText('R coeff = %s, pval = %s' % (R, p), loc=2)
            ax.add_artist(anchored_text)
            fit = np.polyfit(score, feat, deg=1)
            ax.plot(sorted(score), fit[0]*np.array(sorted(score)) + fit[1], color='red')
            ax.scatter(score, feat, color = 'dodgerblue')
            plt.savefig(path2save+'Correlation_'+freq+'_'+scores_names[i]+'_'+feats_names[j]+'.png')
            plt.savefig(path2save+'Correlation_'+freq+'_'+scores_names[i]+'_'+feats_names[j]+'.pdf')
            plt.clf()
            plt.close()
        else:
            continue